In [13]:
from sqlalchemy import create_engine
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from env_vars import *
import pandas as pd
import sqlite3
from sqlalchemy import engine
import pickle
import numpy as np

In [14]:
def authorize():
    client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID,client_secret=CLIENT_SECRET)
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID,client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

def get_features(trackID):
        return sp.audio_features(trackID)[0]

def get_popularity(trackID):
    return sp.track(trackID)['popularity']

def get_markets(trackID):
    foo = sp.track(trackID)['available_markets']
    if foo != None:
        return 1
    else:
        return 0

In [15]:
def get_markets(trackID):
    foo = sp.track(trackID)['available_markets']
    if foo != []:
        return 1
    else:
        return 0

In [16]:
con = sqlite3.connect("song_list_v3_raw.db")

In [17]:
# Load the data into a DataFrame
X = pd.read_sql_query("SELECT * from songs", con)
con.close()

In [18]:
X = X.drop('level_0', axis=1)

In [19]:
X.head(1)

,index,acousticness,artist,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,songid,speechiness,tempo,time_signature,track,valence,genres,popularity,availability
0,0,0.0013,Karkkiautomaatti,0.487,157307.0,0.678,0.0551,9.0,0.0846,-7.78,1.0,5PS5dpaLogPzYU9hWiWyZb,0.0495,149.94,4.0,Tanssi vaan,0.87,"['finnish indie', 'suomi rock']",0,1


In [20]:
engine = create_engine('sqlite:///song_list_v3_22.db', echo=False)
X.to_sql('songs', con=engine)
engine.execute('SELECT * FROM songs;').fetchone()

(0, 0, 0.0013, 'Karkkiautomaatti', 0.487, 157307.0, 0.678, 0.0551, 9.0, 0.0846, -7.78, 1.0, '5PS5dpaLogPzYU9hWiWyZb', 0.0495, 149.94, 4.0, 'Tanssi vaan', 0.87, "['finnish indie', 'suomi rock']", '0', '1')

In [ ]:
for row_index in X['songid'][500000:574019]:
    songid = row_index
    #print(songid)
    popularity = get_popularity(songid)
    #print(popularity)
    markets = get_markets(songid)
    #print(markets)
    try:
        engine.execute(f'UPDATE songs SET popularity = "{popularity}", availability = "{markets}" WHERE songid = \"{songid}\"')
        authorize()
    except:
        print(f"couldn't insert {markets} or {popularity} into table")
        continue